<h2 id="Contents">Contents<a href="#Contents"></a></h2>
        <ol>
        <ol><li><a class="" href="#CASE-Statements">CASE Statements</a></li>
<ol><li><a class="" href="#Basic-Case-Synanx">Basic Case Synanx</a></li>
</ol><li><a class="" href="#Simple-Subqueries">Simple Subqueries</a></li>
<ol><li><a class="" href="#Subqueries-in-WHERE">Subqueries in WHERE</a></li>
<ol><li><a class="" href="#Filtering-using-a-subquery-with-a-scalar">Filtering using a subquery with a scalar</a></li>
<li><a class="" href="#Filtering-using-a-subquery-with-a-list">Filtering using a subquery with a list</a></li>
</ol><li><a class="" href="#Subqueries-in-FROM">Subqueries in FROM</a></li>
<li><a class="" href="#Subqueries-in-SELECT">Subqueries in SELECT</a></li>
<li><a class="" href="#Multiple-Subqueries">Multiple Subqueries</a></li>
</ol><li><a class="" href="#Complex-Subqueries">Complex Subqueries</a></li>
<ol><li><a class="" href="#Correlated-Subqueries">Correlated Subqueries</a></li>
</ol><li><a class="" href="#Nested-Subqueries">Nested Subqueries</a></li>
<ol><li><a class="" href="#Nested-simple-subqueries">Nested simple subqueries</a></li>
<li><a class="" href="#Nest-a-subquery-in-FROM">Nest a subquery in FROM</a></li>
<li><a class="" href="#Common-Table-Expressions">Common Table Expressions</a></li>
<ol><li><a class="" href="#Common-Table-Expressions">Common Table Expressions</a></li>
<li><a class="" href="#Readability">Readability</a></li>
<li><a class="" href="#Organizing-with-CTEs">Organizing with CTEs</a></li>
<li><a class="" href="#CTEs-with-nested-subqueries">CTEs with nested subqueries</a></li>
<li><a class="" href="#Which-Method-to-Use?">Which Method to Use?</a></li>
</ol></ol><li><a class="" href="#Window-Functions">Window Functions</a></li>
<ol><li><a class="" href="#OVER()">OVER()</a></li>
<li><a class="" href="#OVER-and-PARTITION-BY">OVER and PARTITION BY</a></li>
<li><a class="" href="#Sliding-windows">Sliding windows</a></li>
<ol><li><a class="" href="#Slide-to-the-left">Slide to the left</a></li>
<li><a class="" href="#Slide-to-the-right">Slide to the right</a></li>
<li><a class="" href="#Final-Exercise">Final Exercise</a></li>
</ol>


## CASE Statements

A `CASE` statement is SQl version of 'if this then that' statements.<br>
`CASE` statements can be used to:
* Categorizing data
* Filtering data
* Aggregating data

### Basic Case Synanx

<figure>
<figcaption><p>Basic CASE Syntax</p><f/igcaption>
<img src="img/02_01.png"/>
</figure>

>SELECT date<br>
	CASE WHEN home_goal > away_goal THEN 'Home win!'<br>
        WHEN home_goal < away_goal THEN 'Home loss :(' <br>
        ELSE 'Tie' END AS outcome<br>
FROM matches_spain;<br>

The above query uses the `CASE` statement to create a new column called `outcome` that contains the outcome of the match.

It is possible to combine multiple logics inside a `CASE` statement. The following query check if the team is *Chelsea* and classifies the outcome only when this is true!

<figure>
<figcaption><p>CASE Syntax With AND</p><f/igcaption>
<img src="img/02_02.png"/>
</figure>

It is important to add the final `WHEN` condition otherwise the `CASE` statement will output the result coressponding to all teams, not only *Chelsea*.

If we don't want the final `ELSE` clause, we can leave out the `ELSE` clause and it will result in a value of `NULL` everywhere the `ELSE` clause was supposed to return. (In above case, there will be no *Loss or tie :(* in reslting table, instead, it will be `NULL`.). However, we can do better and drop all these `NULL` values using a `WHERE` clause. The syntax is:

<figure>
<figcaption><p>CASE Syntax With AND and WHERE</p><f/igcaption>
<img src="img/02_03.png"/>
</figure>

> We don't need to filter anymore. ):

>SELECT <br>
	date,<br>
	CASE WHEN hometeam_id = 8634 THEN 'FC Barcelona' <br>
         ELSE 'Real Madrid CF' END as home,<br>
	CASE WHEN awayteam_id = 8634 THEN 'FC Barcelona' <br>
         ELSE 'Real Madrid CF' END as away,<br>
	CASE WHEN home_goal > away_goal AND hometeam_id = 8634 THEN 'Barcelona win!'<br>
        WHEN home_goal > away_goal AND hometeam_id = 8633 THEN 'Real Madrid win!'<br>
        WHEN home_goal < away_goal AND awayteam_id = 8634 THEN 'Barcelona win!'<br>
        WHEN home_goal < away_goal AND awayteam_id = 8633 THEN 'Real Madrid win!'<br>
        ELSE 'Tie!' END AS outcome<br>
FROM matches_spain<br>
WHERE (awayteam_id = 8634 OR hometeam_id = 8634)<br>
      AND (awayteam_id = 8633 OR hometeam_id = 8633);<br>

The above query returns the result of all the matches between *Barcelona* and *Real Madrid*. The result is in column `outcome` and has three different values: `Barcelona win!`, `Real Madrid win!` and `Tie!`.

>`CASE` statements allow you to categorize data that you're interested in - and exclude data you're not interested in. In order to do this, you can use a `CASE` statement as a filter in the `WHERE` statement to remove output you don't want to see.

>SELECT <br>
	season,<br>
    date,<br>
	home_goal,<br>
	away_goal<br>
FROM matches_italy<br>
WHERE <br>
	CASE WHEN hometeam_id = 9857 AND home_goal > away_goal THEN 'Bologna Win'<br>
		WHEN awayteam_id = 9857 AND away_goal > home_goal THEN 'Bologna Win' <br>
		END IS NOT NULL;

The above query gives details on every match in this database where *Bologna* won. It excludes matches where *Bologna* didn't win.

`CASE` statements are like any column in our database. So, we can use `COUNT` on them to use `GROUP BY`. The syntax is:

<figure>
<figcaption><p>CASE Syntax With COUNT and GROUP BY</p><f/igcaption>
<img src="img/02_04.png"/>
</figure>

> Note that SQL counts the number of rows that match the `CASE` statement. So, you use any number or string between `THEN` and `END` to group the data.

In case the `CASE` statement returns a number, we can use `SUM` to sum the returned values.

<figure>
<figcaption><p>CASE Syntax With SUM</p><f/igcaption>
<img src="img/02_05.png"/>
</figure>

> The `AVG` can be used the same way.

Another use of `AVG` function is to calculate percentages. The syntax is:

<figure>
<figcaption><p>CASE Syntax With AVG</p><f/igcaption>
<img src="img/02_06.png"/>
</figure>

Here, the query returns the percentage of matches where the team won. We have used the `ROUND` function to round the result to two decimal places.

>SELECT <br>
	c.name AS country,<br>
	COUNT(CASE WHEN m.season = '2012/2013' THEN m.id END) AS matches_2012_2013,<br>
	COUNT(CASE WHEN m.season = '2013/2014' THEN m.id END) AS matches_2013_2014,<br>
	COUNT(CASE WHEN m.season = '2014/2015' THEN m.id END) AS matches_2014_2015<br>
FROM country AS c<br>
LEFT JOIN match AS m<br>
ON c.id = m.country_id<br>
GROUP BY c.name;

The above query gives the number of matches played by countries in year 2012-13, 2013-14 and 2014-15.

>In R or Python, you have the ability to calculate a `SUM` of logical values (i.e., TRUE/FALSE) directly. In SQL, you have to convert these values into 1 and 0 before calculating a sum. This can be done using a `CASE` statement.

>SELECT <br>
	c.name AS country,<br>
	SUM(CASE WHEN m.season = '2012/2013' AND m.home_goal > m.away_goal <br>
        THEN 1 ELSE 0 END) AS matches_2012_2013,<br>
 	SUM(CASE WHEN m.season = '2013/2014' AND m.home_goal > m.away_goal <br>
        THEN 1 ELSE 0 END) AS matches_2013_2014,<br>
	SUM(CASE WHEN m.season = '2014/2015' AND m.home_goal > m.away_goal <br>
        THEN 1 ELSE 0 END) AS matches_2014_2015<br>
FROM country AS c<br>
LEFT JOIN match AS m<br>
ON c.id = m.country_id<br>
GROUP BY c.name;

Here, we have converted the boolean `TRUE` and `FALSE` values into 1 and 0 and summed it.

`CASE` statements will return any value you specify in your `THEN` clause. This is an incredibly powerful tool for robust calculations and data manipulation when used in conjunction with an aggregate statement. One key task you can perform is using `CASE` inside an `AVG` function to calculate a percentage of information in your database.
>AVG(CASE WHEN condition_is_met THEN 1<br>
         WHEN condition_is_not_met THEN 0 END)

>SELECT <br>
	c.name AS country,<br>
	ROUND(AVG(CASE WHEN m.season='2013/2014' AND m.home_goal = m.away_goal THEN 1<br>
			 WHEN m.season='2013/2014' AND m.home_goal != m.away_goal THEN 0<br>
			 END),2) AS pct_ties_2013_2014,<br>
	ROUND(AVG(CASE WHEN m.season='2014/2015' AND m.home_goal = m.away_goal THEN 1<br>
			 WHEN m.season='2014/2015' AND m.home_goal != m.away_goal THEN 0<br>
			 END),2) AS pct_ties_2014_2015<br>
FROM country AS c<br>
LEFT JOIN matches AS m<br>
ON c.id = m.country_id<br>
GROUP BY country;

The above query gives the percentage of tied matches. It also rounds off the result to two decimal places.

## Simple Subqueries

### Subqueries in `WHERE`

#### Filtering using a subquery with a scalar

Subqueries are incredibly powerful for performing complex filters and transformations. You can filter data based on single, scalar values using a subquery in ways you cannot by using `WHERE` statements or joins. Subqueries can also be used for more advanced manipulation of your data set.

>SELECT <br>
    date,<br>
	home_goal,<br>
	away_goal<br>
FROM  matches_2013_2014<br>
WHERE (home_goal + away_goal) <br>
       (SELECT 3 * AVG(home_goal + away_goal)<br>
        FROM matches_2013_2014); <br>

The above query filters only those matches where the total goals scored by the teams are greater than 3 times the average of the total goals scored by the teams in the 2013-14 season.

#### Filtering using a subquery with a list

In addition to filtering using a single-value (scalar) subquery, you can create a list of values in a subquery to filter data based on a complex set of conditions. This type of subquery generates a one column reference list for the main query. As long as the values in your list match a column in your main query's table, you don't need to use a join -- even if the list is from a separate table.

>SELECT <br>
	team_long_name,<br>
	team_short_name<br>
FROM team <br>
WHERE team_api_id NOT IN<br>
     (SELECT DISTINCT hometeam_ID  FROM match);

Here, using a subquery, first, you will generate a list of unique hometeam_ID values from the unfiltered match table to exclude in the team table's team_api_ID column.

We can even create more complex subqueries.

### Subqueries in `FROM`

If you're interested in filtering data from one table, you can also create a subquery from one of the tables, and then join it to an existing table in the database. A subquery in `FROM` is an effective way of answering detailed questions that requires filtering or transforming data before including it in your final results. A subquery in `FROM` returns a table of data that you can then use in your main query.

<figure>
<!-- <figcaption><h4>'CASE Syntax With SUM'</h4><f/igcaption> -->
<img src="img/02_07.png"/>
</figure>

>SELECT<br>
    c.name AS country_name,<br>
    COUNT(sub.id) AS matches<br>
FROM country AS c<br>
INNER JOIN (SELECT country_id, id <br>
           FROM match<br>
           WHERE (home_goal + away_goal) >= 10) AS sub<br>
ON c.id = sub.country_id<br>
GROUP BY country_name;

The above query generates a subquery using the match table, and then join that subquery to the country table to calculate information about matches with 10 or more goals in total! We can get more information about the matches with 10 or more goals in total by:

>SELECT<br>
    country,<br>
    date,<br>
    home_goal,<br>
    away_goal<br>
FROM <br>
	(SELECT c.name AS country, <br>
     	    m.date, <br>
     		m.home_goal, <br>
     		m.away_goal,<br>
           (m.home_goal + m.away_goal) AS total_goals<br>
    FROM match AS m<br>
    LEFT JOIN country AS c<br>
    ON m.country_id = c.id) AS subq<br>
WHERE total_goals >= 10;

### Subqueries in `SELECT`

Subqueries in `SELECT` statements generate a single value that allow you to pass an aggregate value down a data frame. This is useful for performing calculations on data within your database.

<figure>
<figcaption><p>Subqueries in SELECT</p><f/igcaption>
<img src="img/02_08.png"/>
</figure>

>SELECT <br>
	l.name AS league,<br>
    ROUND(AVG(m.home_goal + m.away_goal), 2) AS avg_goals,<br>
    (SELECT ROUND(AVG(home_goal + away_goal), 2) <br>
     FROM match<br>
     WHERE season = '2013/2014') AS overall_avg<br>
FROM league AS l<br>
LEFT JOIN match AS m<br>
ON l.country_id = m.country_id<br>
WHERE season = '2013/2014'<br>
GROUP BY league;

Above query calculates the average number of goals per match in each country's league.

Subqueries in `SELECT` are a useful way to create calculated columns in a query. A subquery in `SELECT` can be treated as a single numeric value to use in your calculations. When writing queries in `SELECT`, it's important to remember that **filtering the main query does not filter the subquery -- and vice versa.**

### Multiple Subqueries

Subqueries can be used in multiple ways. You can use a subquery in `WHERE` to filter data, and then use a subquery in `FROM` to join the data. You can use a subquery in `SELECT` to generate a single value that you can use in your calculations. These all can also be done in a single query.

>SELECT <br>
	s.stage,<br>
	ROUND(s.avg_goals,2) AS avg_goals<br>
FROM <br>
	(SELECT<br>
		 stage,<br>
         AVG(home_goal + away_goal) AS avg_goals<br>
	 FROM match<br>
	 WHERE season = '2012/2013'<br>
	 GROUP BY stage) AS s<br>
WHERE <br>
	s.avg_goals > (SELECT AVG(home_goal + away_goal) <br>
                    FROM match WHERE season = '2012/2013');

The above subquery to extracts a list of stages where the average home goals in a stage is higher than the overall average for home goals in a match.

>SELECT <br>
	s.stage,<br>
    ROUND(s.avg_goals,2) AS avg_goal,<br>
    (SELECT AVG(home_goal + away_goal) FROM match WHERE season = '2012/2013') AS overall_avg<br>
FROM <br>
	(SELECT<br>
		 stage,<br>
         AVG(home_goal + away_goal) AS avg_goals<br>
	 FROM match<br>
	 WHERE season = '2012/2013'<br>
	 GROUP BY stage) AS s<br>
WHERE <br>
	s.avg_goals > (SELECT AVG(home_goal + away_goal) <br>
                    FROM match WHERE season = '2012/2013');

Here, we add a subquery in SELECT, in the previous query to compare the average number of goals scored in each stage to the total. This is an example of a query where we have used subqueries in multiple places (`WHERE`, `FROM`, `SELECT`).

## Complex Subqueries

### Correlated Subqueries

Correlated subqueries are subqueries that reference one or more columns in the main query. Correlated subqueries depend on information in the main query to run, and thus, cannot be executed on their own.

Correlated subqueries are evaluated in SQL once per row of data retrieved - a process that takes a lot more computing power and time than a simple subquery.

![](img/02_09.png)

Difference between correlated subqueries and simple subqueries:

![](img/02_10.png)

## Nested Subqueries

### Nested simple subqueries

A nested subquery is a subquery that is used in another subquery. Nested subqueries can be either simple or correlated.
Just like an unnested subquery, a nested subquery's components can be executed independently of the outer query, while a correlated subquery requires both the outer and inner subquery to run and produce results.

>SELECT<br>
	season,<br>
    MAX(home_goal + away_goal) AS max_goals,<br>
   (SELECT max(home_goal + away_goal) FROM match) AS overall_max_goals,<br>
   (SELECT max(home_goal + away_goal)<br>
    FROM match<br>
    WHERE id IN (<br>
          SELECT id FROM match WHERE EXTRACT(MONTH FROM date) = 07)) AS july_max_goals<br>
FROM match<br>
GROUP BY season;

This subquery gives the highest total number of goals in each season, overall, and during July across all seasons.

### Nest a subquery in `FROM`

Nesting subqueries and performing your transformations one step at a time, adding it to a subquery, and then performing the next set of transformations is often the easiest way to yield accurate information about your data.

SELECT<br>
	c.name AS country,<br>
	AVG(outer_s.matches) AS avg_seasonal_high_scores<br>
FROM country AS c<br>
LEFT JOIN (<br>
  SELECT country_id, season,<br>
         COUNT(id) AS matches<br>
  FROM (<br>
    SELECT country_id, season, id<br>
	FROM match<br>
	WHERE home_goal >= 5 OR away_goal >= 5) AS inner_s<br>
  GROUP BY country_id, season) AS outer_s<br>
ON c.id = outer_s.country_id<br>
GROUP BY country;<br>

The query answers the question: "What is the average number of matches in each country that have a home or away goal greater than or equal to 5?"

### Common Table Expressions

Common table expressions (CTEs) are a way to create a temporary table in your database that can be used in a query. These are used to increase the readability and accessibility of query.

<figure>
<figcaption><h4></h4></figcaption>
<img src ="img/02_11.png">
</figure>

![](img/02_12.png)

The above figure shows the query to create a temporary table. If we need more than one cte, we can use the `WITH` keyword to create multiple ctes, seperated by commas.

![](img/02_13.png)

#### Readability

>WITH match_list AS (<br>
    SELECT <br>
  		country_id, <br>
  		id<br>
    FROM match<br>
    WHERE (home_goal + away_goal) >= 10)<br>
SELECT<br>
    l.name AS league,<br>
    COUNT(match_list.id) AS matches<br>
FROM league AS l<br>
LEFT JOIN match_list ON l.id = match_list.country_id<br>
GROUP BY l.name;

The above query generated a list of countries and the number of matches in each country with more than 10 total goals using a CTE to create a temporary table.

#### Organizing with CTEs

>WITH match_list AS (<br>
    SELECT <br>
  		l.name AS league, <br>
     	date, <br>
  		home_goal, <br>
  		m.away_goal,<br>
       (m.home_goal + m.away_goal) AS total_goals<br>
    FROM match AS m<br>
    LEFT JOIN league as l ON m.country_id = l.id)<br>
SELECT league, date, home_goal, away_goal<br>
FROM match_list<br>
WHERE total_goals >=10;<br>

This query gives details about matches with very high scores using CTEs. Just like a subquery in FROM, you can join tables inside a CTE.

#### CTEs with nested subqueries

Since many queries are written with the intention of being saved and re-run in the future, proper organization is key to a seamless workflow. Arranging subqueries as CTEs will save you time, space, and confusion in the long run!

>WITH match_list AS (<br>
    SELECT <br>
  		country_id,<br>
  	   (home_goal + away_goal) AS goals<br>
    FROM match<br>
    WHERE id IN (<br>
       SELECT id<br>
       FROM match<br>
       WHERE season = '2013/2014' AND EXTRACT(MONTH FROM date) = '08'))<br><br>
SELECT <br>
	l.name,<br>
    AVG(goals)<br>
FROM league AS l<br>
-- Join the CTE onto the league table<br>
LEFT JOIN match_list ON l.id = match_list.country_id<br>
GROUP BY l.name;<br>

As you can see, CTE with a nested subquery is easy to read - and produces an interesting set of data! <br>
Furthermore, as CTEs are evaluated in the starting of the query, they can be used in any part of the query.

#### Which Method to Use?

There are alot of way to get the information we can want from a query. However, they all are not the same. The methods used above, all have some limitations and benefits.

![](img/02_14.png)

Let's answer one single question using all these methods. The question is: How do you get both the home and away team names into one final query result?

>SELECT<br>
	m.date,<br>
    hometeam,<br>
    awayteam,<br>
    m.home_goal,<br>
    m.away_goal<br>
FROM match AS m<br><br>
LEFT JOIN (<br>
  SELECT match.id, team.team_long_name AS hometeam<br>
  FROM match<br>
  LEFT JOIN team<br>
  ON match.hometeam_id = team.team_api_id) AS home<br>
ON home.id = m.id<br><br>
LEFT JOIN (<br>
  SELECT match.id, team.team_long_name AS awayteam<br>
  FROM match<br>
  LEFT JOIN team<br>
  ON match.awayteam_id = team.team_api_id) AS away<br>
ON away.id = m.id;

Here, we are using just `JOIN`. This is one way to generate a list of team names who played in a match. This is much easier to read than team ID numbers!

Let's solve the same problem using correlated subqueries.

>SELECT<br>
    m.date,<br>
    (SELECT team_long_name<br>
     FROM team AS t<br>
     WHERE t.team_api_id = m.hometeam_id) AS hometeam,<br>
    (SELECT team_long_name<br>
     FROM team AS t<br>
     WHERE t.team_api_id = m.awayteam_id) AS awayteam,<br>
     home_goal,<br>
     away_goal<br>
FROM match AS m;

Finally, let's explore the final method - common table expressions.

>WITH home AS (<br>
  SELECT m.id, m.date, <br>
  		 t.team_long_name AS hometeam, m.home_goal<br>
  FROM match AS m<br>
  LEFT JOIN team AS t <br>
  ON m.hometeam_id = t.team_api_id),<br><br>
away AS (<br>
  SELECT m.id, m.date, <br>
  		 t.team_long_name AS awayteam, m.away_goal<br>
  FROM match AS m<br>
  LEFT JOIN team AS t <br>
  ON  m.awayteam_id = t.team_api_id)<br><br>
SELECT <br>
	home.date,<br>
    home.hometeam,<br>
    away.awayteam,<br>
    home.home_goal,<br>
    away.away_goal<br>
FROM home<br>
INNER JOIN away<br>
ON home.id = away.id;

## Window Functions

Window functions are a class of functions that perform calculations on a result set that has already been generated, also referred to as a "window". You can use window functions to perform aggregate calculations without having to group your data, just as you did with a subquery in `SELECT`. You can also use them to calculate information such as running totals, rankings, and moving averages.

![](img/02_15.png)

### `OVER()`

The `OVER()` clause allows you to pass an aggregate function down a data set, similar to subqueries in `SELECT`. The `OVER()` clause offers significant benefits over subqueries in select - namely, your queries will run faster, and the `OVER()` clause has a wide range of additional functions and clauses you can include with it that we will cover later on in this chapter.

>SELECT <br>
	m.id, <br>
    c.name AS country, <br>
    m.season,<br>
	m.home_goal,<br>
	m.away_goal,<br>
	AVG(m.home_goal + m.away_goal) OVER() AS overall_avg<br>
FROM match AS m<br>
LEFT JOIN country AS c ON m.country_id = c.id;<br>

The above query calculates the average number of goals scored overall and then includes the aggregate value in each row using a window function.

Window functions allow you to create a RANK of information according to any variable you want to use to sort your data. When setting this up, you will need to specify what column/calculation you want to use to calculate your rank. This is done by including an `ORDER BY` clause inside the `OVER()` clause.

>SELECT <br>
	l.name AS league,<br>
    AVG(m.home_goal + m.away_goal) AS avg_goals,<br>
    RANK() OVER(ORDER BY AVG(m.home_goal + m.away_goal)) AS league_rank<br>
FROM league AS l<br>
LEFT JOIN match AS m <br>
ON l.id = m.country_id<br>
WHERE m.season = '2011/2012'<br>
GROUP BY l.name<br>
ORDER BY league_rank;

The query above RANKs the average number of goals scored by each league in ascending order.

To get this in decending order, you can use:

>SELECT <br>
	l.name AS league,<br>
    AVG(m.home_goal + m.away_goal) AS avg_goals,<br>
    RANK() OVER(ORDER BY AVG(m.home_goal + m.away_goal) DESC) AS league_rank<br>
FROM league AS l<br>
LEFT JOIN match AS m <br>
ON l.id = m.country_id<br>
WHERE m.season = '2011/2012'<br>
GROUP BY l.name<br>
ORDER BY league_rank;<br>

### `OVER` and `PARTITION BY`

A partition allows you to calculate separate values for different categories established in a partition. This is one way to calculate different aggregate values within one column of data, and pass them down a data set, instead of having to calculate them in different columns. You can also use PARTITION to calculate values broken out by multiple columns.

![](img/02_16.png)

>SELECT<br>
	date,<br>
	season,<br>
	home_goal,<br>
	away_goal,<br>
	CASE WHEN hometeam_id = 8673 THEN 'home' <br>
		 ELSE 'away' END AS warsaw_location,<br>
    AVG(home_goal) OVER(PARTITION BY season) AS season_homeavg,<br>
    AVG(away_goal) OVER(PARTITION BY season) AS season_awayavg<br>
FROM match<br>
WHERE <br>
	hometeam_id = 8673 <br>
    OR awayteam_id = 8673 <br>
ORDER BY (home_goal + away_goal) DESC;

The `PARTITION BY` clause can be used to break out window averages by multiple data points (columns). You can even calculate the information you want to use to partition your data!

>SELECT <br>
	date,<br>
	season,<br>
	home_goal,<br>
	away_goal,<br>
	CASE WHEN hometeam_id = 8673 THEN 'home' <br>
         ELSE 'away' END AS warsaw_location,<br>
    AVG(home_goal) OVER(PARTITION BY season, <br>
         	EXTRACT(month FROM date)) AS season_mo_home,<br>
    AVG(away_goal) OVER(PARTITION BY season, <br>
         	EXTRACT(month FROM date)) AS season_mo_away<br>
FROM match<br>
WHERE 
	hometeam_id = 8673<br>
    OR awayteam_id = 8673 <br>
ORDER BY (home_goal + away_goal) DESC;

### Sliding windows

Sliding windows are functions that perform calculations relative to the current row of a data set. You can use sliding windows to calculate a wide variety of information that aggregates one row at a time down your data set - running totals, sums, counts, and averages in any order you need. A sliding window function contains specific functions within the OVER clause to specify the data you want to use in your calculations. Some of the keywords are:


![](img/02_18.png)

![](img/02_17.png)

#### Slide to the left
Sliding windows allow you to create running calculations between any two points in a window using functions such as PRECEDING, FOLLOWING, and CURRENT ROW. You can calculate running counts, sums, averages, and other aggregate functions between any two points you specify in the data set.

>SELECT <br>
	date,<br>
	home_goal,<br>
	away_goal,<br>
    SUM(home_goal) OVER(ORDER BY date <br>
         ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total,<br>
    AVG(home_goal) OVER(ORDER BY date <br>
         ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)  AS running_avg<br>
FROM match<br>
WHERE <br>
	hometeam_id = 9908 <br>
	AND season = '2011/2012';<br>

The above query calculates the running total and average goals scored by the FC Utrecht team in 2011/2012.

#### Slide to the right

Here, we will slightly modify the query from the previous exercise by sorting the data set in reverse order and calculating a backward running total from the CURRENT ROW to the end of the data set (earliest record).

> SELECT <br>
	date,<br>
    home_goal,<br>
    away_goal,<br>
    SUM(home_goal) OVER(ORDER BY date DESC<br>
         ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS running_total,<br>
    AVG(home_goal) OVER(ORDER BY date DESC<br>
         ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS running_avg<br>
FROM match<br>
WHERE <br>
	awayteam_id = 9908 <br>
    AND season = '2011/2012';

#### Final Exercise

Here, we will be using all of the concepts listed above to generate a list of matches in which Manchester United was defeated during the 2014/2015 English Premier League season.

1. Step I

Create the first query that filters for matches where Manchester United played as the home team. This will become a common table expression in a later exercise.

>SELECT <br>
	m.id, <br>
    t.team_long_name,<br>
    -- Identify matches as home/away wins or ties<br>
	CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'<br>
		WHEN m.home_goal < m.away_goal THEN 'MU Loss'<br>
        ELSE 'Tie' END AS outcome<br>
FROM match AS m<br>
-- Left join team on the home team ID and team API id<br>
LEFT JOIN team AS t <br>
ON m.hometeam_id = t.team_api_id<br>
WHERE <br>
	-- Filter for 2014/2015 and Manchester United as the home team<br>
	m.season = '2014/2015'<br>
	AND t.team_long_name = 'Manchester United';

2. Step II

We'll to a similar query to get the matches where Manchester United played as the away team.

>SELECT <br>
	m.id, <br>
    t.team_long_name,<br>
    -- Identify matches as home/away wins or ties<br>
	CASE WHEN m.home_goal < m.away_goal THEN 'MU Win'<br>
		WHEN m.home_goal > m.away_goal THEN 'MU Loss'<br>
        ELSE 'Tie' END AS outcome<br>
FROM match AS m<br>
-- Left join team on the home team ID and team API id<br>
LEFT JOIN team AS t <br>
ON m.awayteam_id = t.team_api_id<br>
WHERE <br>
	-- Filter for 2014/2015 and Manchester United as the home team<br>
	m.season = '2014/2015'<br>
	AND t.team_long_name = 'Manchester United';

3. Step III

We'll use the above two queries a common table expression to get a list of matches played by Manchester United during the 2014/2015 English Premier League season.

>-- Set up the home team CTE<br>
WITH home AS (<br>
  SELECT m.id, t.team_long_name,<br>
	  CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'<br>
		   WHEN m.home_goal < m.away_goal THEN 'MU Loss' <br>
  		   ELSE 'Tie' END AS outcome<br>
  FROM match AS m<br>
  LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),<br>
-- Set up the away team CTE<br>
away AS (<br>
  SELECT m.id, t.team_long_name,<br>
	  CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'<br>
		   WHEN m.home_goal < m.away_goal THEN 'MU Win' <br>
  		   ELSE 'Tie' END AS outcome<br>
  FROM match AS m<br>
  LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)<br>
-- Select team names, the date and goals<br>
SELECT DISTINCT<br>
    m.date,<br>
    home.team_long_name AS home_team,<br>
    away.team_long_name AS away_team,<br>
    m.home_goal, m.away_goal<br>
-- Join the CTEs onto the match table<br>
FROM match AS m<br>
LEFT JOIN home ON m.id = home.id<br>
LEFT JOIN away ON m.id = away.id<br>
WHERE m.season = '2014/2015'<br>
      AND (home.team_long_name = 'Manchester United' <br>
           OR away.team_long_name = 'Manchester United');

4. Step IV

Here, we'll answer the question: how badly did Manchester United lose in each match?

>-- Set up the home team CTE<br>
WITH home AS (<br>
  SELECT m.id, t.team_long_name,<br>
	  CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'<br>
		   WHEN m.home_goal < m.away_goal THEN 'MU Loss' <br>
  		   ELSE 'Tie' END AS outcome<br>
  FROM match AS m<br>
  LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),<br>
-- Set up the away team CTE<br>
away AS (<br>
  SELECT m.id, t.team_long_name,<br>
	  CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'<br>
		   WHEN m.home_goal < m.away_goal THEN 'MU Win' <br>
  		   ELSE 'Tie' END AS outcome<br>
  FROM match AS m<br>
  LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)<br>
-- Select team names, the date and goals<br>
SELECT DISTINCT<br>
    m.date,<br>
    home.team_long_name AS home_team,<br>
    away.team_long_name AS away_team,<br>
    m.home_goal, m.away_goal,<br>
    RANK() OVER(ORDER BY ABS(home_goal - away_goal) DESC) as match_rank<br>
-- Join the CTEs onto the match table<br>
FROM match AS m<br>
LEFT JOIN home ON m.id = home.id<br>
LEFT JOIN away ON m.id = away.id<br>
WHERE m.season = '2014/2015'<br>
      AND ((home.team_long_name = 'Manchester United' AND home.outcome = 'MU Loss')<br>
      OR (away.team_long_name = 'Manchester United' AND away.outcome = 'MU Loss'));